# 0. Installing Transformers and Importing Dependencies

In [1]:
!pip install transformers

In [48]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 1. Load Summarization Pipeline

In [49]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# 2. Get Blog Post from Medium

In [44]:
# URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
# URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
URL = "https://medium.com/@ai-club/ml101-introduction-to-the-world-of-machine-learning-2-86e32c740b58"

In [47]:
URL

'https://medium.com/@ai-club/ml101-introduction-to-the-world-of-machine-learning-2-86e32c740b58'

In [50]:
r = requests.get(URL)

In [52]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [51]:
r

<Response [200]>

In [54]:
len(ARTICLE)

7172

In [60]:
print(ARTICLE)

Sign up Sign in Sign up Sign in ML101: Introduction to the world of Machine Learning — 2 PTU AI CLUB Follow -- Listen Share Authors: Harsha, Sakti Priya V, Yuvarani VD and Apurva Introduction Our last article in the ML101 series by PTU AI CLUB introduced you to the world of machine learning and its infinite use cases.<eos> In this article we will be diving a little deeper into this world by exploring the types of machine learning and the ethical aspects related to machine learning.<eos> We will also be providing you with a roadmap for getting started in this world.<eos> Enough talk, let’s ML!<eos> Types of Machine Learning Based on the methods and way of learning, machine learning is divided into mainly four types, which are: 1.<eos> Supervised Machine Learning2.<eos> Unsupervised Machine Learning3.<eos> Semi-supervised Machine Learning4.<eos> Reinforcement Learning Supervised machine learning is based on supervision.<eos> It means in the supervised learning technique, we train the mac

# 3. Chunk Text

In [61]:
max_chunk = 500

In [62]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [63]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [64]:
len(chunks)

3

# 4. Summarize Text

In [65]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [66]:
res[0]

{'summary_text': 'the first article in the ML101 series by PTU AI CLUB introduces you to the world of machine learning and its infinite use cases . in this article we will be diving a little deeper into this world by exploring the types of machines and the ethical aspects related to machine learning . the concept of semi-supervised learning is introduced .'}

In [67]:
' '.join([summ['summary_text'] for summ in res])

'the first article in the ML101 series by PTU AI CLUB introduces you to the world of machine learning and its infinite use cases . in this article we will be diving a little deeper into this world by exploring the types of machines and the ethical aspects related to machine learning . the concept of semi-supervised learning is introduced . semi-supervised learning is where a student is under the supervision of an instructor . if that student is self-analyzing the same concept, it comes under unsupervised learning . the goal of reinforcement learning agent is to maximize the rewards . machine learning has the power to revolutionize the way we live our lives and interact with the world around us . a world where financial fraud is detected and prevented in real-time, making our financial systems more secure than ever before . it’s important to consider the potential ethical implications and risks associated with machine learning .'

In [68]:
text = ' '.join([summ['summary_text'] for summ in res])

# 5. Output to Text File

In [69]:
text

'the first article in the ML101 series by PTU AI CLUB introduces you to the world of machine learning and its infinite use cases . in this article we will be diving a little deeper into this world by exploring the types of machines and the ethical aspects related to machine learning . the concept of semi-supervised learning is introduced . semi-supervised learning is where a student is under the supervision of an instructor . if that student is self-analyzing the same concept, it comes under unsupervised learning . the goal of reinforcement learning agent is to maximize the rewards . machine learning has the power to revolutionize the way we live our lives and interact with the world around us . a world where financial fraud is detected and prevented in real-time, making our financial systems more secure than ever before . it’s important to consider the potential ethical implications and risks associated with machine learning .'

In [70]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)